<a href="https://colab.research.google.com/github/INT2-Team2/NeuralNetwork/blob/Adam/INT2%20Group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%%bash
pip install captum
pip install flask_compress
conda install freetype=2.10.4

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/first_visualization_test')

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance
from captum.insights import AttributionVisualizer, Batch
from captum.insights.attr_vis.features import ImageFeature
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
if torch.cuda.is_available(): # use gpu if possible
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
epochs = 40
batch_size = 16
learning_rate = 0.0001

# only need resize so AlexNet works
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.RandomHorizontalFlip(),
                                transforms.Normalize((0.5, 0.5, 0.5), 
                                                     (0.5, 0.5, 0.5))])

In [ ]:
# get training/test data from CIFAR10 dataset
train_data = torchvision.datasets.CIFAR10(root = "./dataset", 
                                        train = True, 
                                        transform = transform, 
                                        download = True)
test_data = torchvision.datasets.CIFAR10(root = "./dataset", 
                                       train = False, 
                                       transform = transform, 
                                       download = True)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           num_workers = 2)
test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size = batch_size, 
                                          shuffle = False, 
                                          num_workers = 2)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 128, 3, 1)
    self.conv2 = nn.Conv2d(128, 256, 3, 1, padding=2)
    self.conv3 = nn.Conv2d(256, 256, 3, 1, padding=2)
    self.conv4 = nn.Conv2d(256, 512, 3, 1, padding=1)
    self.conv5 = nn.Conv2d(512, 512, 3, 1, padding=1)
    self.conv6 = nn.Conv2d(512, 1024, 3, 1)
    self.conv7 = nn.Conv2d(1024, 1024, 3, 1)

    self.maxPool = nn.MaxPool2d(2, 2)

    self.drop1 = nn.Dropout(0.3)
    self.drop2 = nn.Dropout(0.3)
    self.drop3 = nn.Dropout(0.2)
    self.drop4 = nn.Dropout(0.4)

    self.fc1 = nn.Linear(1024, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 10)
  
  def forward(self, x):
    i = 0
    f = lambda i: [i+1,print(i,x.shape)][0]


    x = nn.functional.rrelu(self.conv1(x))
    x = self.drop1(x)
    x = self.maxPool(x)
    x = nn.functional.rrelu(self.conv2(x))
    x = self.drop2(x)
    x = self.maxPool(x)
    x = nn.functional.rrelu(self.conv3(x))
    x = nn.functional.rrelu(self.conv4(x))
    x = self.drop3(x)
    x = self.maxPool(x)
    x = nn.functional.rrelu(self.conv5(x))
    x = nn.functional.rrelu(self.conv6(x))
    x = self.drop4(x)
    x = nn.functional.rrelu(self.conv7(x))
    #i=f(i)
    x = x.reshape(x.shape[0], -1)
    #i=f(i)
    x = nn.functional.rrelu(self.fc1(x))
    x = nn.functional.rrelu(self.fc2(x))
    x = self.fc3(x)
    #i=f(i)
    return x
  

In [ ]:
# Model creation

model_copy = CNN()
model = CNN().to(device)

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    try:
      npimg = img.numpy()
    except:
      npimg = img.cpu().data.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().data.numpy())
    return preds, [nn.functional.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            train_data.classes[preds[idx]],
            probs[idx] * 100.0,
            train_data.classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig


In [ ]:
def test():
  print("Testing")
  with torch.no_grad():
    correct = 0
    samples = 0

    for (images, labels) in test_loader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      _, predictions = outputs.max(1)

      samples += labels.size(0)
      correct += (predictions == labels).sum()

    print("Test accuracy was",100*float(correct)/float(samples))
    print()

In [ ]:
## Training

# try other loss functions/optimizers
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=0.0015)

test_per_epoch = True
train_for_time = 0 # how many minutes to train for (and then finish current epoch)

if train_for_time:
  epochs = train_for_time*1000
start = time.time()

for epoch in range(epochs):
  print("Training")
  epoch_loss = 0
  previous_loss = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forwards
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backwards
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

    if i % 1000 == 999:    # every 1000 mini-batches...

            # ...log the running loss
            writer.add_scalar('training loss',
                            epoch_loss-previous_loss / 1000,
                            epoch * len(train_loader) + i)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_figure('predictions vs. actuals',
                            plot_classes_preds(model, images, labels),
                            global_step=epoch * len(train_loader) + i)
            previous_loss = epoch_loss
  
  print("Epoch", epoch+1, "complete")
  print("Loss was", epoch_loss/len(train_loader))
  print()
  if test_per_epoch:
    test()
  
  if train_for_time and time.time()-start >= train_for_time*60:
    break

if not test_per_epoch:
  test()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def baseline_func(input):
  return 0

visualizer = AttributionVisualizer(
    models=[CNN],
    score_func=lambda o: torch.nn.functional.softmax(o, 1),
    classes= train_data.classes,
    features=[
        ImageFeature(
            "Photo",
            baseline_transforms=[baseline_func],
            input_transforms=[transform],
        )
    ],
    dataset=test_data,
)

In [ ]:
visualizer.render()

In [ ]:
visualizer.serve()

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
writer.add_graph(model_copy, images)
writer.add_image('first_visualization_test', img_grid)
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/first_visualization_test/